## Select Latest Dataset

In [0]:
spark.conf.set("###",dbutils.secrets.get(scope = "###", key = "###"))

In [0]:
filenamesmonthlydata=dbutils.fs.ls("MembershipMonthlySourceData/")
filenameschurndates=dbutils.fs.ls("MembershipChurnDatesSourceData/")

In [0]:
filelistmonthlydata=[file.name for file in filenamesmonthlydata if 'PublishMembershipMonthly' in file.name]
filelistchurndates=[file.name for file in filenameschurndates if 'MemberChurnDates' in file.name]

In [0]:
filedatesmonthlydata=[name.split('_')[1] for name in filelistmonthlydata]
filedateschurndates=[name.split('_')[1] for name in filelistchurndates]
filedatesmonthlydata=[name.split('.')[0] for name in filedatesmonthlydata]
filedateschurndates=[name.split('.')[0] for name in filedateschurndates]

In [0]:
import pandas as pd
import databricks.koalas as ks
pdfiledatesmonthlydata=pd.Series(filedatesmonthlydata)
pdfiledateschurndates=pd.Series(filedateschurndates)
pdfiledatesmonthlydata = pd.to_datetime(pdfiledatesmonthlydata, infer_datetime_format=True)
pdfiledateschurndates = pd.to_datetime(pdfiledateschurndates, infer_datetime_format=True)

In [0]:
maxdatemonthlydata=str(pdfiledatesmonthlydata.max()).replace(' ','T').replace(':','H',1).replace(':','m',1)+'s'
maxdatechurndates=str(pdfiledateschurndates.max()).replace(' ','T').replace(':','H',1).replace(':','m',1)+'s'

In [0]:
latestfilemonthlydata='PublishMembershipMonthly_'+maxdatemonthlydata+'.parquet'
latestfilechurndates='MemberChurnDates_'+maxdatechurndates+'.parquet'

In [0]:
churnthreshdate=pdfiledatesmonthlydata.max().date()
churnthreshdate2=ks.Series([pd.Timestamp(churnthreshdate)]).astype(str)[0]

## Load two tables from ADLS

In [0]:
dfcloseddate=spark.read.parquet("MembershipChurnDatesSourceData/"+latestfilechurndates)

In [0]:
dftrain=spark.read.parquet("MembershipMonthlySourceData/"+latestfilemonthlydata)

In [0]:
import databricks.koalas as ks
dfchange2 = ks.DataFrame(dftrain)
dfchange2['EOMDate']= ks.to_datetime(dfchange2['EOMDate'],infer_datetime_format=True)

In [0]:
dftrain = dfchange2.to_spark()

## Join two Tables

In [0]:
dfcloseddate = dfcloseddate.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
df = dftrain.join(dfcloseddate, dftrain.MemberIdentifier == dfcloseddate.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## Separating train and set datasets

In [0]:
import databricks.koalas as ks 
dfk = ks.DataFrame(df)

In [0]:
dfk['ClosedDate']= dfk['ClosedDate'].astype(str)

In [0]:
import pandas as pd 
# x = pd.Timestamp(year=2020, month=5, day=21)
x = churnthreshdate

In [0]:
condition = dfk['ClosedDate'].isna()

#function mask() to change the value of the column 
change_column = dfk['ClosedDate'].mask(condition,x)

#create a new df with the modified ratings 
dftest10 = dfk.assign(ClosedDate=change_column)

In [0]:
dfk['EOMDate']= ks.to_datetime(dfk['EOMDate'],infer_datetime_format=True)
dftest10['ClosedDate']=ks.to_datetime(dftest10['ClosedDate'],infer_datetime_format=True)

In [0]:
df3 = dftest10.to_spark()

## Creating Target Variable

In [0]:
dfktarget = ks.DataFrame(df3)

In [0]:
dfktarget['ClosedDate'] = dfktarget['ClosedDate'].astype(str)

In [0]:
dfktarget['TARGET'] = dfktarget['ClosedDate']

In [0]:
import numpy as np 

condition = dfktarget['TARGET'] == churnthreshdate2

#function mask() to change the value of the column 
change_column = dfktarget['TARGET'].mask(condition,0)

#create a new df with the modified ratings 
df4 = dfktarget.assign(TARGET=change_column)

In [0]:
condition = df4['TARGET'] != 0

#function mask() to change the value of the column 
change_column = df4['TARGET'].mask(condition,1)

#create a new df with the modified ratings 
df5 = df4.assign(TARGET=change_column)

In [0]:
dftrain2 = df5.to_spark()

#### Months Elapsed

In [0]:
dfk =ks.DataFrame(dftrain2)

In [0]:
dfk['ClosedDate']= ks.to_datetime(dfk['ClosedDate'],infer_datetime_format=True) 
dfk['EOMDate']= ks.to_datetime(dfk['EOMDate'],infer_datetime_format=True)

In [0]:
dfk['months_elapsed']=dfk['ClosedDate']-dfk['EOMDate']

In [0]:
dfk['months_elapsed']=(dfk['months_elapsed']/2592000)-1

In [0]:
dfk3 = dfk[dfk.months_elapsed <= 4]

In [0]:
dfk3 = dfk3[dfk3.months_elapsed >= 0]

In [0]:
dfk3['months_elapsed']=dfk3['months_elapsed'].astype(int)

In [0]:
dfk3['months_elapsed'] = dfk3['months_elapsed'].astype(str) + "_month_ago"

##Pivot Table

In [0]:
dfk3['HouseholdIdentifier'] =  dfk3['HouseholdIdentifier'].astype(int)

In [0]:
dfk3['DaysSinceEngaged'] =  dfk3['DaysSinceEngaged'].astype(int)

In [0]:
dfk3['TARGET'] =  dfk3['TARGET'].astype(int)

In [0]:
df = dfk3.to_spark()

In [0]:
dffinal = df.groupBy("MemberIdentifier").pivot("months_elapsed").sum(
"HouseholdIdentifier",
"DebitCountShare",
"CreditCountShare",
"DebitAmountShare",
"CreditAmountShare",
"DebitCountLoan",
"CreditCountLoan",
"DebitAmountLoan",
"CreditAmountLoan",
"CountOfInstrumentConditionBankruptcy",
"CountOfInstrumentConditionInactive",
"CountOfInstrumentConditionClosedByInstitution",
"CountOfInstrumentStatusClosed",
"CountOfInstrumentStatusOpen",
"CountOfPaymentStatusChargedOff",
"CountOfPaymentStatusChargedOffPaidinFull",
"CountOfPaymentStatusNegativeShareBalance",
"CountOfPaymentStatusPaidOff",
"HasOnlineBanking",
"HasNoMarketing",
"HasCheckingRewards",
"HasDirectDeposit",
"HasFPSInvestment",
"HasNoMarketingMail",
"POELoans",
"POERealEstate",
"POESavings",
"POETransactional",
"POEWealth",
"DaysSinceEngaged",
"ProdCountPOETransactional",
"ProdCountPOEWealth",
"ProdCountPOERealEstate",
"ProdCountPOESavings",
"ProdCountPOELoans",
"ProdAmountPOETransactional",
"ProdAmountPOEWealth",
"ProdAmountPOERealEstate",
"ProdAmountPOESavings",
"ProdAmountPOELoans",
"NoOfTransactionsBranch",
"NoOfTransactionsOperations",
"NoOfTransactionsContactCenter",
"NoOfTransactionsIndirectLending",
"NoOfTransactionsSharedBranch",
"NoOfTransactionsMobileOnlineBanking",
"NoOfTransactionsSystemGenerated",
"NoOfTransactionsElectronicClearing",
"NoOfTransactionsMerchantSign",
"NoOfTransactionsLockbox",
"TARGET"
)

In [0]:
dffinalKs = ks.DataFrame(dffinal)

In [0]:
columnslist=dffinalKs.columns

In [0]:
columnslist2 = [sub.replace('sum(CAST(', '') for sub in columnslist]
columnslist2 = [sub.replace(' AS BIGINT))', '') for sub in columnslist2]
columnslist2 = [sub.replace(')', '') for sub in columnslist2]
columnslist2 = [sub.replace('sum(', '') for sub in columnslist2]

In [0]:
dffinalKs.columns = columnslist2

In [0]:
dffinal=dffinalKs.to_spark()

In [0]:
dffinal = dffinal.withColumnRenamed('0_month_ago_TARGET','TARGET')

In [0]:
columns_to_drop = ['1_month_ago_TARGET', '2_month_ago_TARGET','3_month_ago_TARGET','4_month_ago_TARGET']
dffinal = dffinal.drop(*columns_to_drop)

In [0]:
dffinal.write.mode("overwrite").format("com.databricks.spark.parquet").parquet("Retraining/TransformedData/TransformedData_"+maxdatemonthlydata+".parquet")